In [ ]:
import urllib
from urllib.request import build_opener
import urllib3
import json
from http import cookiejar
from urllib.request import HTTPCookieProcessor
from urllib.request import install_opener
import re
import requests
from bs4 import BeautifulSoup
import pandas
import numpy as np
# 我们这里发现了链家翻页的时候，后面的pg会+1，所以我们会输入相应的页数来达到爬取多页面的行为

base_url = "https://gz.lianjia.com/ershoufang/pg{}/"
# 构造headers，这些都是在chrome的开发者工具里面获取来的
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Cookie': 'select_city=110000; all-lj=dafad6dd721afb903f2a315ab2f72633; lianjia_uuid=20e04e6a-43b9-4aff-92dd-da1a6ee8ac5d; TY_SESSION_ID=5c0be168-287f-4a70-81ef-27814df5724b; _smt_uid=5ca9b45b.c2b1829; sajssdk_2015_cross_new_user=1; _ga=GA1.2.1478954141.1554625630; _gid=GA1.2.1048010799.1554625630; Hm_lvt_9152f8221cb6243a53c83b956842be8a=1554625628,1554626037; lianjia_ssid=4318f646-8d65-40f1-8079-6daa47bdd643; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%22169f6e8889a2f9-09a6f2c0908db2-e323069-1327104-169f6e8889b7d4%22%2C%22%24device_id%22%3A%22169f6e8889a2f9-09a6f2c0908db2-e323069-1327104-169f6e8889b7d4%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%7D%7D; _gat=1; _gat_past=1; _gat_global=1; _gat_new_global=1; _gat_dianpu_agent=1; Hm_lpvt_9152f8221cb6243a53c83b956842be8a=1554626134',
    'Host': 'bj.lianjia.com',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}

# 我们传入参数page，指定爬虫的页数
def generate_url(page):
    
    for url_next in range(1, page+1):
        yield base_url.format(url_next)
        # 构造一个生成器，生成多页面url
        
# 访问我们的url，并正则表达式匹配到每一个住房的具体页面的url
![](https://img-blog.csdnimg.cn/20190408150456322.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80NDEwNjkyOA==,size_16,color_FFFFFF,t_70)
# 访问的就是这些带有标题的页面
def getallurl(generate_url):
    html = requests.get(generate_url, headers=headers)
    if html.status_code == 200:
        re_set = re.compile('<a.*?class="noresultRecommend.*?img.*?".*?href="(.*?)"')
        re_get = re.findall(re_set, html.text)
        return re_get
# 利用beautifsoup选择我们需要的数据，相关数字我们进行int转换，并以列表的形式添加进去
def open_url(re_get, info_index, info_title, info_price, info_unit):
    res = requests.get(re_get, headers=headers)
    if res.status_code == 200:
        info = {}
        soup = BeautifulSoup(res.text, 'lxml')
        info_index.append(int(float(soup.select('.total')[0].text)))
        info_title.append(soup.select('.main')[0].text)
        info_price.append(soup.select('.total')[0].text + '万')
        info_unit.append(soup.select('.unitPrice')[0].text)
        info_total = zip(info_index, info_title, info_price, info_unit )
 # 为了返回结果方便，我先zip了这四个数据合并到一起，后续再unpack
    return  info_total

# 这个方法就是unpack
def unzip_elements(info_elements):
    index, title, price, unit = [*zip(*info_elements)]
    index_list = list(index)
    title_list = list(title)
    price_list = list(price)
    unit_list = list(unit)
    return index_list, title_list, price_list, unit_list
# 构造一个dataFrame来实现写入excel的操作
def save_to_xlsx(info):
    pdlook = pandas.DataFrame(info)
    pdlook.to_excel(city +'链家.xlsx', sheet_name="链家二手房"+city)
# 主程序
if __name__ == '__main__':
    page = int(input("请输入你要的页数"))
    city = str(input("请输入你要城市"))
    if city == '北京':
        base_url = "https://bj.lianjia.com/ershoufang/pg{}/"
    if city == '上海':
        base_url = "https://sh.lianjia.com/ershoufang/pg{}/"
    elif city == '广州':
        base_url = "https://gz.lianjia.com/ershoufang/pg{}/"
    elif city == '成都':
        base_url = "https://cd.lianjia.com/ershoufang/pg{}/"
    # 定义要存储数据的列表
    info_index = []
    info_title = []
    info_price = []
    info_unit = []
    # 这里我用2个for循环实现获取页面url，获取每一个页面的28个具体住房信息url
    for i in generate_url(page):
        print(i)
        get = getallurl(i)
        print(len(get))
        # 因为每个页面有28个结果
        for i in range (1, 30):
            info_elements = open_url(get[i], info_index, info_title, info_price, info_unit)

        # 将数据unpack，分别存到这四个变量中
        index_list, title_list, price_list, unit_list = unzip_elements(info_elements)
        # 将各个数据组合成一个字典
        data = {'index': index_list, 'title':title_list, 'price':price_list, 'unit':unit_list}
        save_to_xlsx(data)
        print(data)


/bin/sh: -c: line 0: syntax error near unexpected token `https://img-blog.csdnimg.cn/20190408150456322.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80NDEwNjkyOA==,size_16,color_FFFFFF,t_70'
/bin/sh: -c: line 0: `[](https://img-blog.csdnimg.cn/20190408150456322.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80NDEwNjkyOA==,size_16,color_FFFFFF,t_70)'
请输入你要的页数100
请输入你要城市成都
https://cd.lianjia.com/ershoufang/pg1/
30
{'index': [700, 649, 440, 276, 225, 405, 632, 475, 540, 390, 870, 258, 430, 726, 445, 395, 425, 420, 531, 630, 275, 480, 560, 480, 710, 333, 450, 388, 319], 'title': ['黑窑厂西里南北通透低楼层两居室', '爱民里  满五年  一居 随时可看', '月季园 2室1厅 440万', '全南向一居室，明厨明卫，中间楼层，视野好，采光充足', '通州北苑商圈 满五唯一 全明户型 看房方便', '商品房满五年唯一  小区中间位置  采光好', '格兰晴天 3室2厅 632万', '沁春家园精装修南北向两居室采光无遮挡业主诚心出售', '板楼南向精装两居室 户型方正使用率高 看房方便', '模式口北里 2室2厅 390万', '满五年唯一，南北通透三居室，婚房装修', '三环边儿富顿劲松地铁口低总价南向开间诚意出售', '板楼的2层，小区安

30
{'index': [700, 649, 440, 276, 225, 405, 632, 475, 540, 390, 870, 258, 430, 726, 445, 395, 425, 420, 531, 630, 275, 480, 560, 480, 710, 333, 450, 388, 319, 435, 410, 290, 325, 368, 475, 326, 355, 220, 919, 377, 635, 499, 495, 1020, 460, 610, 499, 449, 280, 378, 567, 513, 520, 635, 438, 520, 329, 505, 482, 435, 535, 258, 685, 282, 415, 398, 429, 636, 820, 925, 578, 998, 649, 480, 688, 399, 550, 449, 610, 325, 462, 419, 748, 252, 320, 357, 359, 690, 329, 645, 729, 468, 329, 490, 585, 528, 460, 1450, 425, 1020, 793, 495, 920, 1368, 250, 470, 640, 260, 370, 550, 408, 296, 666, 375, 460, 418], 'title': ['黑窑厂西里南北通透低楼层两居室', '爱民里  满五年  一居 随时可看', '月季园 2室1厅 440万', '全南向一居室，明厨明卫，中间楼层，视野好，采光充足', '通州北苑商圈 满五唯一 全明户型 看房方便', '商品房满五年唯一  小区中间位置  采光好', '格兰晴天 3室2厅 632万', '沁春家园精装修南北向两居室采光无遮挡业主诚心出售', '板楼南向精装两居室 户型方正使用率高 看房方便', '模式口北里 2室2厅 390万', '满五年唯一，南北通透三居室，婚房装修', '三环边儿富顿劲松地铁口低总价南向开间诚意出售', '板楼的2层，小区安静整洁，税费少，随时签约', '满五年不唯一公房，诚心出售，户型方正。', '世纪村西区 满五唯一 精装南北两居室 中间层', '回龙观精装修电梯两居两卫 南北通透 一梯两户', '满五唯一无个税，板楼二层，南

30
{'index': [700, 649, 440, 276, 225, 405, 632, 475, 540, 390, 870, 258, 430, 726, 445, 395, 425, 420, 531, 630, 275, 480, 560, 480, 710, 333, 450, 388, 319, 435, 410, 290, 325, 368, 475, 326, 355, 220, 919, 377, 635, 499, 495, 1020, 460, 610, 499, 449, 280, 378, 567, 513, 520, 635, 438, 520, 329, 505, 482, 435, 535, 258, 685, 282, 415, 398, 429, 636, 820, 925, 578, 998, 649, 480, 688, 399, 550, 449, 610, 325, 462, 419, 748, 252, 320, 357, 359, 690, 329, 645, 729, 468, 329, 490, 585, 528, 460, 1450, 425, 1020, 793, 495, 920, 1368, 250, 470, 640, 260, 370, 550, 408, 296, 666, 375, 460, 418, 825, 415, 540, 890, 650, 386, 500, 468, 375, 210, 375, 230, 280, 509, 373, 758, 478, 559, 580, 420, 450, 335, 375, 359, 575, 955, 435, 588, 348, 316, 298, 880, 630, 340, 435, 686, 420, 423, 560, 352, 635, 457, 298, 375, 568, 456, 265, 750, 555, 1250, 920, 435, 255, 389, 520, 440, 330, 240], 'title': ['黑窑厂西里南北通透低楼层两居室', '爱民里  满五年  一居 随时可看', '月季园 2室1厅 440万', '全南向一居室，明厨明卫，中间楼层，视野好，采光充足', '通州北苑商圈 满五唯一 全

30
{'index': [700, 649, 440, 276, 225, 405, 632, 475, 540, 390, 870, 258, 430, 726, 445, 395, 425, 420, 531, 630, 275, 480, 560, 480, 710, 333, 450, 388, 319, 435, 410, 290, 325, 368, 475, 326, 355, 220, 919, 377, 635, 499, 495, 1020, 460, 610, 499, 449, 280, 378, 567, 513, 520, 635, 438, 520, 329, 505, 482, 435, 535, 258, 685, 282, 415, 398, 429, 636, 820, 925, 578, 998, 649, 480, 688, 399, 550, 449, 610, 325, 462, 419, 748, 252, 320, 357, 359, 690, 329, 645, 729, 468, 329, 490, 585, 528, 460, 1450, 425, 1020, 793, 495, 920, 1368, 250, 470, 640, 260, 370, 550, 408, 296, 666, 375, 460, 418, 825, 415, 540, 890, 650, 386, 500, 468, 375, 210, 375, 230, 280, 509, 373, 758, 478, 559, 580, 420, 450, 335, 375, 359, 575, 955, 435, 588, 348, 316, 298, 880, 630, 340, 435, 686, 420, 423, 560, 352, 635, 457, 298, 375, 568, 456, 265, 750, 555, 1250, 920, 435, 255, 389, 520, 440, 330, 240, 355, 660, 385, 345, 248, 450, 399, 310, 1100, 373, 606, 460, 498, 540, 459, 325, 630, 538, 280, 739, 420, 478, 

30
{'index': [700, 649, 440, 276, 225, 405, 632, 475, 540, 390, 870, 258, 430, 726, 445, 395, 425, 420, 531, 630, 275, 480, 560, 480, 710, 333, 450, 388, 319, 435, 410, 290, 325, 368, 475, 326, 355, 220, 919, 377, 635, 499, 495, 1020, 460, 610, 499, 449, 280, 378, 567, 513, 520, 635, 438, 520, 329, 505, 482, 435, 535, 258, 685, 282, 415, 398, 429, 636, 820, 925, 578, 998, 649, 480, 688, 399, 550, 449, 610, 325, 462, 419, 748, 252, 320, 357, 359, 690, 329, 645, 729, 468, 329, 490, 585, 528, 460, 1450, 425, 1020, 793, 495, 920, 1368, 250, 470, 640, 260, 370, 550, 408, 296, 666, 375, 460, 418, 825, 415, 540, 890, 650, 386, 500, 468, 375, 210, 375, 230, 280, 509, 373, 758, 478, 559, 580, 420, 450, 335, 375, 359, 575, 955, 435, 588, 348, 316, 298, 880, 630, 340, 435, 686, 420, 423, 560, 352, 635, 457, 298, 375, 568, 456, 265, 750, 555, 1250, 920, 435, 255, 389, 520, 440, 330, 240, 355, 660, 385, 345, 248, 450, 399, 310, 1100, 373, 606, 460, 498, 540, 459, 325, 630, 538, 280, 739, 420, 478, 

...

30
{'index': [700, 649, 440, 276, 225, 405, 632, 475, 540, 390, 870, 258, 430, 726, 445, 395, 425, 420, 531, 630, 275, 480, 560, 480, 710, 333, 450, 388, 319, 435, 410, 290, 325, 368, 475, 326, 355, 220, 919, 377, 635, 499, 495, 1020, 460, 610, 499, 449, 280, 378, 567, 513, 520, 635, 438, 520, 329, 505, 482, 435, 535, 258, 685, 282, 415, 398, 429, 636, 820, 925, 578, 998, 649, 480, 688, 399, 550, 449, 610, 325, 462, 419, 748, 252, 320, 357, 359, 690, 329, 645, 729, 468, 329, 490, 585, 528, 460, 1450, 425, 1020, 793, 495, 920, 1368, 250, 470, 640, 260, 370, 550, 408, 296, 666, 375, 460, 418, 825, 415, 540, 890, 650, 386, 500, 468, 375, 210, 375, 230, 280, 509, 373, 758, 478, 559, 580, 420, 450, 335, 375, 359, 575, 955, 435, 588, 348, 316, 298, 880, 630, 340, 435, 686, 420, 423, 560, 352, 635, 457, 298, 375, 568, 456, 265, 750, 555, 1250, 920, 435, 255, 389, 520, 440, 330, 240, 355, 660, 385, 345, 248, 450, 399, 310, 1100, 373, 606, 460, 498, 540, 459, 325, 630, 538, 280, 739, 420, 478, 

30
{'index': [700, 649, 440, 276, 225, 405, 632, 475, 540, 390, 870, 258, 430, 726, 445, 395, 425, 420, 531, 630, 275, 480, 560, 480, 710, 333, 450, 388, 319, 435, 410, 290, 325, 368, 475, 326, 355, 220, 919, 377, 635, 499, 495, 1020, 460, 610, 499, 449, 280, 378, 567, 513, 520, 635, 438, 520, 329, 505, 482, 435, 535, 258, 685, 282, 415, 398, 429, 636, 820, 925, 578, 998, 649, 480, 688, 399, 550, 449, 610, 325, 462, 419, 748, 252, 320, 357, 359, 690, 329, 645, 729, 468, 329, 490, 585, 528, 460, 1450, 425, 1020, 793, 495, 920, 1368, 250, 470, 640, 260, 370, 550, 408, 296, 666, 375, 460, 418, 825, 415, 540, 890, 650, 386, 500, 468, 375, 210, 375, 230, 280, 509, 373, 758, 478, 559, 580, 420, 450, 335, 375, 359, 575, 955, 435, 588, 348, 316, 298, 880, 630, 340, 435, 686, 420, 423, 560, 352, 635, 457, 298, 375, 568, 456, 265, 750, 555, 1250, 920, 435, 255, 389, 520, 440, 330, 240, 355, 660, 385, 345, 248, 450, 399, 310, 1100, 373, 606, 460, 498, 540, 459, 325, 630, 538, 280, 739, 420, 478, 

30
{'index': [700, 649, 440, 276, 225, 405, 632, 475, 540, 390, 870, 258, 430, 726, 445, 395, 425, 420, 531, 630, 275, 480, 560, 480, 710, 333, 450, 388, 319, 435, 410, 290, 325, 368, 475, 326, 355, 220, 919, 377, 635, 499, 495, 1020, 460, 610, 499, 449, 280, 378, 567, 513, 520, 635, 438, 520, 329, 505, 482, 435, 535, 258, 685, 282, 415, 398, 429, 636, 820, 925, 578, 998, 649, 480, 688, 399, 550, 449, 610, 325, 462, 419, 748, 252, 320, 357, 359, 690, 329, 645, 729, 468, 329, 490, 585, 528, 460, 1450, 425, 1020, 793, 495, 920, 1368, 250, 470, 640, 260, 370, 550, 408, 296, 666, 375, 460, 418, 825, 415, 540, 890, 650, 386, 500, 468, 375, 210, 375, 230, 280, 509, 373, 758, 478, 559, 580, 420, 450, 335, 375, 359, 575, 955, 435, 588, 348, 316, 298, 880, 630, 340, 435, 686, 420, 423, 560, 352, 635, 457, 298, 375, 568, 456, 265, 750, 555, 1250, 920, 435, 255, 389, 520, 440, 330, 240, 355, 660, 385, 345, 248, 450, 399, 310, 1100, 373, 606, 460, 498, 540, 459, 325, 630, 538, 280, 739, 420, 478, 

30
{'index': [700, 649, 440, 276, 225, 405, 632, 475, 540, 390, 870, 258, 430, 726, 445, 395, 425, 420, 531, 630, 275, 480, 560, 480, 710, 333, 450, 388, 319, 435, 410, 290, 325, 368, 475, 326, 355, 220, 919, 377, 635, 499, 495, 1020, 460, 610, 499, 449, 280, 378, 567, 513, 520, 635, 438, 520, 329, 505, 482, 435, 535, 258, 685, 282, 415, 398, 429, 636, 820, 925, 578, 998, 649, 480, 688, 399, 550, 449, 610, 325, 462, 419, 748, 252, 320, 357, 359, 690, 329, 645, 729, 468, 329, 490, 585, 528, 460, 1450, 425, 1020, 793, 495, 920, 1368, 250, 470, 640, 260, 370, 550, 408, 296, 666, 375, 460, 418, 825, 415, 540, 890, 650, 386, 500, 468, 375, 210, 375, 230, 280, 509, 373, 758, 478, 559, 580, 420, 450, 335, 375, 359, 575, 955, 435, 588, 348, 316, 298, 880, 630, 340, 435, 686, 420, 423, 560, 352, 635, 457, 298, 375, 568, 456, 265, 750, 555, 1250, 920, 435, 255, 389, 520, 440, 330, 240, 355, 660, 385, 345, 248, 450, 399, 310, 1100, 373, 606, 460, 498, 540, 459, 325, 630, 538, 280, 739, 420, 478, 

30


In [5]:
import urllib
from urllib.request import build_opener
import urllib3
import json
from http import cookiejar
from urllib.request import HTTPCookieProcessor
from urllib.request import install_opener
import re
import requests
from bs4 import BeautifulSoup
import pandas
import numpy as np

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Cookie': 'select_city=110000; all-lj=dafad6dd721afb903f2a315ab2f72633; lianjia_uuid=20e04e6a-43b9-4aff-92dd-da1a6ee8ac5d; TY_SESSION_ID=5c0be168-287f-4a70-81ef-27814df5724b; _smt_uid=5ca9b45b.c2b1829; sajssdk_2015_cross_new_user=1; _ga=GA1.2.1478954141.1554625630; _gid=GA1.2.1048010799.1554625630; Hm_lvt_9152f8221cb6243a53c83b956842be8a=1554625628,1554626037; lianjia_ssid=4318f646-8d65-40f1-8079-6daa47bdd643; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%22169f6e8889a2f9-09a6f2c0908db2-e323069-1327104-169f6e8889b7d4%22%2C%22%24device_id%22%3A%22169f6e8889a2f9-09a6f2c0908db2-e323069-1327104-169f6e8889b7d4%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_referrer%22%3A%22%22%2C%22%24latest_referrer_host%22%3A%22%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%7D%7D; _gat=1; _gat_past=1; _gat_global=1; _gat_new_global=1; _gat_dianpu_agent=1; Hm_lpvt_9152f8221cb6243a53c83b956842be8a=1554626134',
    'Host': 'bj.lianjia.com',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}

res = requests.get('https://cd.office.fang.com/shou/3_208698538.html?channel=1,32', headers=headers)
    
info = {}
soup = BeautifulSoup(res.text, 'lxml')
#info_index.append(int(float(soup.select('.total')[0].text)))
#info_title.append(soup.select('.main')[0].text)
#info_price.append(soup.select('.total')[0].text + '万')
#info_unit.append(soup.select('.unitPrice')[0].text)
#info_total = zip(info_index, info_title, info_price, info_unit )

In [6]:
soup

<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<center><h1>404 Not Found</h1></center>
<hr/><center>nginx</center>
</body>
</html>
<!-- a padding to disable MSIE and Chrome friendly error page -->

In [15]:
cont ='//cd.ganji.com/shangpu/39249948716067x.shtml'
cont.strip('/')
print(cont.strip('//'))

cd.ganji.com/shangpu/39249948716067x.shtml


In [16]:
from fake_useragent import UserAgent

In [30]:
url = 'http://cd.ganji.com/shangpucs/'

URLs = []

for i in range(1,72):
            
    URLs.append(url + "pn" + str(i) + "/")
    
print(len(URLs))


for i in range(len(URLs)):
    
    print(URLs[i])
    


71
http://cd.ganji.com/shangpucs/pn1/
http://cd.ganji.com/shangpucs/pn2/
http://cd.ganji.com/shangpucs/pn3/
http://cd.ganji.com/shangpucs/pn4/
http://cd.ganji.com/shangpucs/pn5/
http://cd.ganji.com/shangpucs/pn6/
http://cd.ganji.com/shangpucs/pn7/
http://cd.ganji.com/shangpucs/pn8/
http://cd.ganji.com/shangpucs/pn9/
http://cd.ganji.com/shangpucs/pn10/
http://cd.ganji.com/shangpucs/pn11/
http://cd.ganji.com/shangpucs/pn12/
http://cd.ganji.com/shangpucs/pn13/
http://cd.ganji.com/shangpucs/pn14/
http://cd.ganji.com/shangpucs/pn15/
http://cd.ganji.com/shangpucs/pn16/
http://cd.ganji.com/shangpucs/pn17/
http://cd.ganji.com/shangpucs/pn18/
http://cd.ganji.com/shangpucs/pn19/
http://cd.ganji.com/shangpucs/pn20/
http://cd.ganji.com/shangpucs/pn21/
http://cd.ganji.com/shangpucs/pn22/
http://cd.ganji.com/shangpucs/pn23/
http://cd.ganji.com/shangpucs/pn24/
http://cd.ganji.com/shangpucs/pn25/
http://cd.ganji.com/shangpucs/pn26/
http://cd.ganji.com/shangpucs/pn27/
http://cd.ganji.com/shangpucs/pn28